In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),    # 随机水平翻转
    transforms.RandomRotation(15),        # 随机旋转±15度
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 三通道归一化
])

In [4]:
# 加载数据集
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 输入3通道，输出32通道
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)              # 2x2最大池化
        self.fc1 = nn.Linear(64 * 8 * 8, 512)       # 全连接层计算：64*(32/2/2)^2
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.25)             # 防止过拟合

    def forward(self, x):
        x = F.relu(self.conv1(x))   # 32x32x32
        x = self.pool(x)            # 32x16x16
        x = F.relu(self.conv2(x))   # 64x16x16
        x = self.pool(x)            # 64x8x8
        x = torch.flatten(x, 1)     # 展平为64*8*8=4096维
        x = self.dropout(x)
        x = F.relu(self.fc1(x))     # 512维
        x = self.fc2(x)             # 10维输出
        return x

model = CIFAR10Net()

In [6]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [7]:
for epoch in range(25):
    running_loss = 0.0
    model.train()
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    scheduler.step()
    print(f'Epoch {epoch+1} Loss: {running_loss/len(trainloader):.3f}')

Epoch 1 Loss: 1.426
Epoch 2 Loss: 1.117
Epoch 3 Loss: 0.987
Epoch 4 Loss: 0.907
Epoch 5 Loss: 0.839
Epoch 6 Loss: 0.787
Epoch 7 Loss: 0.737
Epoch 8 Loss: 0.703
Epoch 9 Loss: 0.660
Epoch 10 Loss: 0.628
Epoch 11 Loss: 0.538
Epoch 12 Loss: 0.518
Epoch 13 Loss: 0.510
Epoch 14 Loss: 0.498
Epoch 15 Loss: 0.493
Epoch 16 Loss: 0.483
Epoch 17 Loss: 0.480
Epoch 18 Loss: 0.470
Epoch 19 Loss: 0.465
Epoch 20 Loss: 0.462
Epoch 21 Loss: 0.454
Epoch 22 Loss: 0.450
Epoch 23 Loss: 0.449
Epoch 24 Loss: 0.450
Epoch 25 Loss: 0.446


In [8]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 76.66%
